<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/b4_public_datasets_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using public datasets with TF Datasets

In [1]:
!pip install tensorflow-addons

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

from tensorflow.keras import layers

tfds.__version__

'4.0.1'

In [3]:
mnist_data = tfds.load('fashion_mnist')
type(mnist_data), mnist_data

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteO8DE3H/fashion_mnist-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteO8DE3H/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


(dict,
 {'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
  'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>})

In [4]:
for item in mnist_data:
  print(type(item), item)

<class 'str'> test
<class 'str'> train


If you want to load these splits into a dataset containing the actual data, you can simply specify the split you want in the tfds.load command, like this:

In [5]:
mnist_train = tfds.load(name='fashion_mnist', split='train')
assert isinstance(mnist_train, tf.data.Dataset)
type(mnist_train)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In this instance, we we a `PrefetchDataset` object, which we can iterate through to inspect the data. One nice feature is that we can apply `take(1)` and get the first record.

In [6]:
item = next(iter(mnist_train.take(1)))
print(type(item))
print(item.keys())

<class 'dict'>
dict_keys(['image', 'label'])


In [7]:
image = item['image']
print(type(image))
print(image.shape)
print(image[0:0])

<class 'tensorflow.python.framework.ops.EagerTensor'>
(28, 28, 1)
tf.Tensor([], shape=(0, 28, 1), dtype=uint8)


In [8]:
label = item['label']
print(type(label))
print(label)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(2, shape=(), dtype=int64)


In [9]:
mnist_test, info = tfds.load(name='fashion_mnist', with_info='true')
info

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=3.0.1,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   Kashif Rasul and
                   Roland Vollgraf},
      title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
                   Algorithms},
      journal   = {CoRR},
      volume

## Using TFDS with Keras Model

In [10]:
mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(type(train_images))

4423680/4422102 [==============================] - 0s 0us/step
<class 'numpy.ndarray'>


When using TFDS the code is very similar, but with some minor changes. The Keras datasets gave us `ndarray` that worked natively in `model.fit`. However, with TFDS we will need to do a little conversion work.

In [11]:
(train_images, train_labels), (test_images, test_labels) = \
  tfds.as_numpy(
      tfds.load('fashion_mnist',
                split=['train', 'test'],
                batch_size=-1,
                as_supervised=True))
print(type(train_images))

<class 'numpy.ndarray'>


In [12]:
# we need to rescale our images before feeding them into the network
# train_images = train_images * 1.0/255.0
# test_images = test_images * 1.0/255.0
# skipping this rescaling step in favor of adding rescaling directly
# into the model pipeline(see layers...Rescaling) 

model = tf.keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1.0/255.0),
  layers.Flatten(input_shape=(28, 28, 1)),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(10, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.fit(
    train_images,
    train_labels,
    epochs=5
)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5319 - accuracy: 0.8109
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4018 - accuracy: 0.8546
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3679 - accuracy: 0.8661
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3472 - accuracy: 0.8744
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3330 - accuracy: 0.8773


The data is batched and shuffled to make training more effective?

## Human-or-Horses Model

In [13]:
data = tfds.load('horses_or_humans', split='train', as_supervised=True)
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)
 
train_batches = data.shuffle(100).batch(10)
validation_batches = val_data.batch(32)

model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_batches, 
    epochs=10,
    validation_data=validation_batches
)

Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLB2IDV/horses_or_humans-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLB2IDV/horses_or_humans-test.tfrecord


Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.
Epoch 1/10
103/103 [==============================] - 5s 53ms/step - loss: 0.3637 - accuracy: 0.8130 - val_loss: 1.7935 - val_accuracy: 0.8008
Epoch 2/10
103/103 [==============================] - 5s 47ms/step - loss: 0.1105 - accuracy: 0.9611 - val_loss: 1.6182 - val_accuracy: 0.8203
Epoch 3/10
103/103 [==============================] - 5s 47ms/step - loss: 0.0987 - accuracy: 0.9688 - val_loss: 1.3668 - val_accuracy: 0.8789
Epoch 4/10
103/103 [==============================] - 5s 48ms/step - loss: 0.0922 - accuracy: 0.9698 - val_loss: 1.6270 - val_accuracy: 0.8086
Epoch 5/10
103/103 [==============================] - 5s 47ms/step - loss: 0.0181 - accuracy: 0.9961 - val_loss: 1.2874 - val_accuracy: 0.8906
Epoch 6/10
103/103 [==============================] - 5s 47ms/step - loss: 0.0331 - accuracy: 0.9922 - val_loss: 0.9671 - val_accuracy: 0.8789
E

## Using Mapping Functions for Augmentation

In [14]:
def augment_images(image, label):
  image = tf.image.random_flip_left_right(image)
  return image, label

In [15]:
train = data.map(augment_images)

In [16]:
train_batches = train.shuffle(100).batch(32)
model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_batches, 
    epochs=10,
)

Epoch 1/10
33/33 [==============================] - 4s 126ms/step - loss: 0.3687 - accuracy: 0.8286
Epoch 2/10
33/33 [==============================] - 4s 122ms/step - loss: 0.0942 - accuracy: 0.9601
Epoch 3/10
33/33 [==============================] - 4s 124ms/step - loss: 0.1045 - accuracy: 0.9620
Epoch 4/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0552 - accuracy: 0.9844
Epoch 5/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0255 - accuracy: 0.9912
Epoch 6/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0159 - accuracy: 0.9942
Epoch 7/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0402 - accuracy: 0.9844
Epoch 8/10
33/33 [==============================] - 4s 121ms/step - loss: 0.0091 - accuracy: 0.9981
Epoch 9/10
33/33 [==============================] - 4s 123ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 10/10
33/33 [==============================] - 4s 122ms/step - loss: 2.4745e-04 - accuracy: 1.

## Sofisticated Augmentation

In [17]:
def resize(image, label):
  image = tf.image.resize(image, [300, 300]) 
  return image, label 

In [18]:
def augment_image(image, label):
  image, label = resize(image, label)
  # random crop back to the original size
  image = tf.image.random_crop(image, size=[300, 300, 3])
  # random brightness
  image = tf.image.random_brightness(image, max_delta=0.5)
  # image = tfa.image.rotate(image, 40, interpolation='NEAREST')
  image = tf.clip_by_value(image, 0, 1)
  return image, label

In [19]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = (
    train
    .shuffle(1000)
    .map(augment_image, num_parallel_calls=AUTOTUNE)
    .batch(32)
    .prefetch(AUTOTUNE)
)

In [20]:
model = tf.keras.models.Sequential([
    layers.experimental.preprocessing.Rescaling(1.0/255.0),
    layers.Conv2D(16, (3,3), activation='relu', 
                           input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])

history = model.fit(
    train_ds, 
    epochs=10
)

Epoch 1/10
 2/33 [>.............................] - ETA: 2s - loss: 0.6929 - accuracy: 0.4688WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0261s vs `on_train_batch_end` time: 0.0934s). Check your callbacks.


33/33 [==============================] - 2s 46ms/step - loss: 0.6937 - accuracy: 0.5083
Epoch 2/10
33/33 [==============================] - 1s 45ms/step - loss: 0.6930 - accuracy: 0.5131
Epoch 3/10
33/33 [==============================] - 1s 44ms/step - loss: 0.6932 - accuracy: 0.5131
Epoch 4/10
33/33 [==============================] - 1s 44ms/step - loss: 0.6931 - accuracy: 0.5131
Epoch 5/10
33/33 [==============================] - 1s 44ms/step - loss: 0.6930 - accuracy: 0.5131
Epoch 6/10
33/33 [==============================] - 1s 45ms/step - loss: 0.6931 - accuracy: 0.5131
Epoch 7/10
33/33 [==============================] - 1s 45ms/step - loss: 0.6930 - accuracy: 0.5131
Epoch 8/10
33/33 [==============================] - 1s 44ms/step - loss: 0.6930 - accuracy: 0.5131
Epoch 9/10
33/33 [==============================] - 1s 45ms/step - loss: 0.6929 - accuracy: 0.5131
Epoch 10/10
33/33 [==============================] - 1s 44ms/step - loss: 0.6932 - accuracy: 0.5131


## Using Custom Split

Load the first 10K records of `train`.

In [21]:
data = tfds.load('cats_vs_dogs', split='train[:1000]', as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete4DH6GF/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


You can also use % to specify the split. For example, if you want the firtst 20% of the records to be used for training, you could use: 20% like this:

In [22]:
data = tfds.load('cats_vs_dogs', split='train[:20%]')

In [23]:
a = np.arange(10)
a[-5:]

array([5, 6, 7, 8, 9])

In [24]:
a[:5]

array([0, 1, 2, 3, 4])

In [25]:
data = tfds.load('cats_vs_dogs', split='train[-1000:]+train[:1000]')

The Dogs vs. Cats dataset doesn;t have fixed training, test, and validation splits, but, with TFDS, creating your own is simple. Suppose you want the split 80%, 10% and 10%. You could create the three sets like this:

In [27]:
train_data = tfds.load('cats_vs_dogs', split='train[:80%]', as_supervised=True)

validation_data = tfds.load('cats_vs_dogs', split='train[80%:90%]', as_supervised=True)

test_data = tfds.load('cats_vs_dogs', split='train[-10%:]', as_supervised=True)